# Load model

In [ ]:
from huggingface_hub import login
import os

# Load your token from environment variable for security
token = os.getenv("HF_TOKEN")

if token is None:
    raise ValueError("⚠️ No Hugging Face token found. Please set HF_TOKEN in your environment.")

login(token=token)

In [ ]:
%pip -q install accelerate
# %pip -q install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

model = Gemma3ForConditionalGeneration.from_pretrained(
    "google/gemma-3-4b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"
)
processor = AutoProcessor.from_pretrained(
    "google/gemma-3-4b-it",
    padding_side="left"
)

Test the model

In [ ]:
# messages = [
#     {
#         "role": "system",
#         "content": [
#             {"type": "text", "text": "You are a helpful assistant."}
#         ]
#     },
#     {
#         "role": "user", "content": [
#             {"type": "text", "text": "In one word: what is the colour of the sky?"}
#         ]
#     }
# ]
# inputs = processor.apply_chat_template(
#     messages,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
#     add_generation_prompt=True,
# ).to("cuda")

# output = model.generate(**inputs, max_new_tokens=40, eos_token_id=processor.tokenizer.eos_token_id,pad_token_id=processor.tokenizer.pad_token_id, cache_implementation="static")
# print(processor.decode(output[0], skip_special_tokens=True))

In [ ]:
# print(f"Model device: {next(model.parameters()).device}")
# print(f"Input device: {inputs['input_ids'].device}")

# Load database

In [ ]:
%pip install -qU datasets pillow #ignore the error outputs
# Loading the data from Hugging Face
from datasets import load_dataset, load_from_disk
import os

def get_scienceqa_dataset():
      print("Downloading from HuggingFace...")
      dataset = load_dataset("derek-thomas/ScienceQA")
      return dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.2 MB/s eta 0:00:00


In [ ]:
dataset = get_scienceqa_dataset()["train"]
evalset = get_scienceqa_dataset()["validation"]

Loading dataset from disk: /content/drive/MyDrive/comp9444_scienceqa/scienceqa_dataset
Loading dataset from disk: /content/drive/MyDrive/comp9444_scienceqa/scienceqa_dataset


In [ ]:
# #filter out image questions
# def is_text_only(example):
#     return example["image"] is None

# dataset = dataset.filter(is_text_only)
# evalset = evalset.filter(is_text_only)

In [ ]:
# print(f"Filtered dataset size: {len(text_dataset)}")

In [ ]:
from PIL import Image

def format_evaluation(example):
    q = example["question"]
    choices = example["choices"]
    hint = example.get("hint", "")
    lecture = example.get("lecture", "")
    skill = example.get("skill", "")
    image = example.get("image", None)

    user_content = []

    if lecture:
        user_content.append({"type": "text", "text": f"Lecture: {lecture}"})
    if skill:
        user_content.append({"type": "text", "text": f"Skill: {skill}"})
    if isinstance(image, Image.Image):  # ✅ Leave image as PIL.Image
        user_content.append({"type": "image", "image": image})

    user_content.append({"type": "text", "text": f"Question: {q}"})
    user_content.append({
        "type": "text",
        "text": f"Choices: {', '.join([f'{i}. {c}' for i, c in enumerate(choices)])}\nHint: {hint}\nAnswer:"
    })

    messages = [
        {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
        {"role": "user", "content": user_content}
    ]

    return {"messages": messages, "answer": str(example["answer"])}  # <-- useful for evaluation



In [ ]:
subset = dataset.select(range(2))  # first 2 examples

In [ ]:
print(subset[0])
print(format_evaluation(subset[0]))

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7B41FA2135D0>, 'question': 'Which of these states is farthest north?', 'choices': ['West Virginia', 'Louisiana', 'Arizona', 'Oklahoma'], 'answer': 0, 'hint': '', 'task': 'closed choice', 'grade': 'grade2', 'subject': 'social science', 'topic': 'geography', 'category': 'Geography', 'skill': 'Read a map: cardinal directions', 'lecture': 'Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.\nA compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.\nThe north arrow points to the North Pole. On most maps, north is at the top of the map.', 'solution': 'To find the answer, look at the compass rose. Look at which way the north arrow is pointing. West Virginia is farthest north.'}
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful assis

In [ ]:
from tqdm import tqdm

def evaluate_model(model, processor, dataset, batch_size=1, max_samples=None):
    model.eval()
    correct = 0
    total = 0

    # Step 1: Format examples
    if max_samples:
        dataset = dataset.select(range(min(max_samples, len(dataset))))
    formatted_ds = [format_evaluation(ex) for ex in dataset]

    # Step 2: Run in batches
    for i in tqdm(range(0, len(formatted_ds), batch_size), desc="Evaluating"):
        batch = formatted_ds[i:i + batch_size]

        messages_batch = [ex["messages"] for ex in batch]
        answers_batch = [str(ex["answer"]).strip() for ex in batch]

        try:
            # Apply chat template to the whole batch
            inputs = processor.apply_chat_template(
                messages_batch,
                tokenize=True,
                return_tensors="pt",
                return_dict=True,
                add_generation_prompt=True,
            ).to("cuda")

            # Generate predictions
            outputs = model.generate(
                **inputs,
                max_new_tokens=40,
                eos_token_id=processor.tokenizer.eos_token_id,
                pad_token_id=processor.tokenizer.pad_token_id,
                cache_implementation="static"
            )

            # Decode predictions and compare
            decoded = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)
            for pred, true in zip(decoded, answers_batch):
              if pred.strip() == true:
                  correct += 1
              total += 1
        except Exception as e:
            print(f"⚠️ Error at batch {i // batch_size}: {e}")
            total += len(batch)  # count them as attempted

    accuracy = correct / total if total > 0 else 0
    print(f"\n✅ Accuracy: {accuracy:.4f} ({correct}/{total})")
    return accuracy


In [ ]:
accuracy = evaluate_model(model, processor, subset, batch_size=1, max_samples=10)
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
print(type(dataset), type(dataset[0]))

<class 'datasets.arrow_dataset.Dataset'> <class 'dict'>


------

In [ ]:
from PIL import Image
from io import BytesIO

def format_example(example):
    q = example["question"]
    choices = example["choices"]
    hint = example.get("hint", "")
    answer = example["answer"]
    lecture = example.get("lecture", "")
    skill = example.get("skill", "")
    image = example.get("image", None)

    if isinstance(image, Image.Image):
        buffer = BytesIO()
        image.save(buffer, format="PNG")
        image_bytes = buffer.getvalue()
        image = {"bytes": image_bytes}
    else:
        image = None

    user_content = []

    if lecture:
        user_content.append({"type": "text", "text": f"Lecture: {lecture}\n"})
    if skill:
        user_content.append({"type": "text", "text": f"Skill: {skill}\n"})
    if image:
        user_content.append({"type": "image", "bytes": image})
    user_content.append({"type": "text", "text": f"Question: {q}\n"})
    user_content.append({
        "type": "text",
        "text": f"Choices: {', '.join([f'{i}. {c}' for i, c in enumerate(choices)])}\nHint: {hint}\nAnswer:"
    })

    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": [{"type": "text", "text": str(answer)}]}
    ]

    return {"messages": messages}

In [ ]:
from PIL import Image
from io import BytesIO

def format_evaluation(example):
    q = example["question"]
    choices = example["choices"]
    hint = example.get("hint", "")
    lecture = example.get("lecture", "")
    skill = example.get("skill", "")
    image = example.get("image", None)

    if isinstance(image, Image.Image):
        buffer = BytesIO()
        image.save(buffer, format="PNG")
        image_bytes = buffer.getvalue()
        image = {"bytes": image_bytes}
    else:
        image = None

    user_content = []

    if lecture:
        user_content.append({"type": "text", "text": f"Lecture: {lecture}\n"})
    if skill:
        user_content.append({"type": "text", "text": f"Skill: {skill}\n"})
    if image:
        user_content.append({"type": "image", "bytes": image})
    user_content.append({"type": "text", "text": f"Question: {q}\n"})
    user_content.append({
        "type": "text",
        "text": f"Choices: {', '.join([f'{i}. {c}' for i, c in enumerate(choices)])}\nHint: {hint}\nAnswer:"
    })

    messages = [
        {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
        {"role": "user", "content": user_content}
        ]

    return {"messages": messages}

In [ ]:
from tqdm import tqdm

def evaluate_model(model, processor, dataset, max_samples=None):
    model.eval()
    correct = 0
    total = 0

    subset = dataset.select(range(min(max_samples, len(dataset)))) if max_samples else dataset

    for example in tqdm(subset, desc="Evaluating"):
        formatted = format_evaluation(example)
        messages = formatted["messages"]

        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
            add_generation_prompt=True,
        ).to("cuda")

        output = model.generate(
            **inputs,
            max_new_tokens=40,
            eos_token_id=processor.tokenizer.eos_token_id,
            pad_token_id=processor.tokenizer.pad_token_id,
            cache_implementation="static"
        )

        prediction = processor.decode(output[0], skip_special_tokens=True).strip()
        correct_answer = str(example["answer"]).strip()

        if prediction == correct_answer:
            correct += 1
        total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

In [ ]:
#sanity check
example = dataset[0]
formatted = format_example(example)
print(formatted)

{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Lecture: Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.\nA compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.\nThe north arrow points to the North Pole. On most maps, north is at the top of the map.'}, {'type': 'text', 'text': 'Skill: Read a map: cardinal directions'}, {'type': 'image', 'image': {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xee\x00\x00\x01\xad\x08\x02\x00\x00\x00\xc6[f\xc2\x00\x01\x00\x00IDATx\x9c\xec\xfdwt\\\xc9\x9e\xdf\t\xfe"\xae\xcf\x9b\xde\xc2{\x10 @\xefYd\xb1\xc8\xb2,\xfb\xaa\x9e\xed\xee\xa7UKj\xd9\xd1hgGs\xb4\xd2\xee\xce\xb6tfWs\x8eV\xe6\x8c\xb4sf{4#7jI\xef\xf5\xeb\xe7\xca\x1bV\x15\x8b\xde\x93 \x00\x12\xde\xdb\xf4>\xf3\xba\x88\xfd\x03x4 \x08\x9b\x99\x00\xc8\xfb\xf9\xa3\x8a\xb87n\xdc\xb8\x99y#\xbe\x11\xf13\x88R\n\x00\x00\x10\x

In [ ]:
#sanity check
subset = dataset.select(range(2))  # first 2 examples
formatted = subset.map(format_example)

In [ ]:
print(subset[0])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7ECC264A7050>, 'question': 'Which of these states is farthest north?', 'choices': ['West Virginia', 'Louisiana', 'Arizona', 'Oklahoma'], 'answer': 0, 'hint': '', 'task': 'closed choice', 'grade': 'grade2', 'subject': 'social science', 'topic': 'geography', 'category': 'Geography', 'skill': 'Read a map: cardinal directions', 'lecture': 'Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.\nA compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.\nThe north arrow points to the North Pole. On most maps, north is at the top of the map.', 'solution': 'To find the answer, look at the compass rose. Look at which way the north arrow is pointing. West Virginia is farthest north.'}


In [ ]:
print(formatted[0])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7D5C55234410>, 'question': 'Which of these states is farthest north?', 'choices': ['West Virginia', 'Louisiana', 'Arizona', 'Oklahoma'], 'answer': 0, 'hint': '', 'task': 'closed choice', 'grade': 'grade2', 'subject': 'social science', 'topic': 'geography', 'category': 'Geography', 'skill': 'Read a map: cardinal directions', 'lecture': 'Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.\nA compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.\nThe north arrow points to the North Pole. On most maps, north is at the top of the map.', 'solution': 'To find the answer, look at the compass rose. Look at which way the north arrow is pointing. West Virginia is farthest north.', 'messages': [{'content': [{'image': {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0

In [ ]:
dataset = dataset.map(format_example)
evalset = evalset.map(format_example)

In [ ]:
def tokenize_fn(example):
    model_inputs = processor.apply_chat_template(
        example["messages"],
        tokenize=True,
        return_tensors=None,  # Needed for map compatibility
        return_dict=True,
        add_generation_prompt=True,  # Adds assistant turn for generation
        padding=True,  # Add this line
        truncation=True
    )

    # Add labels (required for training)
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs

In [ ]:
tokenized = formatted.map(tokenize_fn, remove_columns=dataset.column_names)

In [ ]:
print(type(tokenized))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
import torch

# Convert attention_mask to a tensor
mask = torch.tensor(tokenized[0]["attention_mask"])

# Print its shape
print("Shape of attention_mask:", mask.shape)

Shape of attention_mask: torch.Size([1, 140])


In [ ]:
output = model.generate(**tokenized, max_new_tokens=40, eos_token_id=processor.tokenizer.eos_token_id,pad_token_id=processor.tokenizer.pad_token_id, cache_implementation="static")
print(processor.decode(output[0], skip_special_tokens=True))

In [ ]:
tokenized_dataset = dataset.map(tokenize_fn, remove_columns=dataset.column_names)
tokenized_evalset = evalset.map(tokenize_fn, remove_columns=evalset.column_names)

Try:

In [ ]:
from PIL import Image
from io import BytesIO

def format_evaluation(example):
    q = example["question"]
    choices = example["choices"]
    hint = example.get("hint", "")
    lecture = example.get("lecture", "")
    skill = example.get("skill", "")
    image = example.get("image", None)

    # Keep image as-is if it's a PIL image
    if not isinstance(image, Image.Image):
        image = None

    user_content = []

    if lecture:
        user_content.append({"type": "text", "text": f"Lecture: {lecture.strip()}"})
    if skill:
        user_content.append({"type": "text", "text": f"Skill: {skill.strip()}"})
    if image:
        user_content.append({"type": "image", "image": image})
    user_content.append({"type": "text", "text": f"Question: {q.strip()}"})
    user_content.append({
        "type": "text",
        "text": f"Choices: {', '.join([f'{i}. {c}' for i, c in enumerate(choices)])}\nHint: {hint.strip()}\nAnswer:"
    })

    messages = [
        {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
        {"role": "user", "content": user_content}
    ]

    return {"messages": messages}


In [ ]:
formatted_ds = [format_evaluation(example) | {"answer": example["answer"]} for example in dataset.select(range(10))]

In [ ]:
def evaluate_model_batch(model, processor, formatted_ds, batch_size=1):
    model.eval()
    correct = 0
    total = 0

    for i in range(0, len(formatted_ds), batch_size):
        batch = formatted_ds[i:i + batch_size]

        messages_batch = [ex["messages"] for ex in batch]
        answers_batch = [str(ex["answer"]).strip() for ex in batch]

        # Tokenize all messages in one go
        inputs = processor.apply_chat_template(
            messages_batch,
            tokenize=True,
            return_tensors="pt",
            return_dict=True,
            add_generation_prompt=True,
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=40,
            eos_token_id=processor.tokenizer.eos_token_id,
            pad_token_id=processor.tokenizer.pad_token_id,
            cache_implementation="static"
        )

        decoded_outputs = [processor.decode(o, skip_special_tokens=True).strip() for o in outputs]

        correct += sum([pred == answer for pred, answer in zip(decoded_outputs, answers_batch)])
        total += len(batch)

    return correct / total if total else 0



In [ ]:
accuracy = evaluate_model_batch(model, processor, formatted_ds)
print(f"Accuracy: {accuracy:.2%}")

Gemma3ForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_war

Accuracy: 0.00%


# Evaluate the pre-trained model

In [ ]:
model.eval()

Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=1152,

In [ ]:
import torch
from PIL import Image
import io

def evaluate_model(model, processor, dataset, max_samples=None):
    model.eval()
    correct = 0
    total = 0
    predictions = []

    for i, example in enumerate(dataset):
        if max_samples and i >= max_samples:
            break

        image = to_pil(example.get("image"))

        prompt = processor.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = processor.tokenizer(
            prompt,
            return_tensors="pt",
            padding="longest",
            truncation=True
        )

        # Add pixel_values if applicable
        if image:
            inputs["pixel_values"] = processor.image_processor(image, return_tensors="pt")["pixel_values"]

        # Move everything to model.device
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=2,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id
            )

        new_tokens = output[0][inputs["input_ids"].shape[-1]:]
        prediction = processor.tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        is_correct = prediction == str(example["answer"])
        correct += is_correct
        total += 1

        predictions.append({
            "gold": str(example["answer"]),
            "pred": prediction,
            "correct": is_correct,
            "used_image": image is not None
        })

    accuracy = correct / total if total > 0 else 0
    return accuracy, predictions

In [ ]:
base_acc, base_preds = evaluate_model(model, processor, evalset)

import json
with open("base_eval_results.json", "w") as f:
    json.dump({
        "accuracy": base_acc,
        "predictions": base_preds
    }, f, indent=2)

print(f"Base model accuracy: {base_acc:.2%}")

# Apply LoRA

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

def multimodal_collator(batch):
    input_ids = pad_sequence(
        [torch.tensor(b["input_ids"]) for b in batch],
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )

    attention_mask = pad_sequence(
        [torch.tensor(b["attention_mask"]) for b in batch],
        batch_first=True,
        padding_value=0
    )

    labels = pad_sequence(
        [torch.tensor(b["labels"]) for b in batch],
        batch_first=True,
        padding_value=-100
    )

    # 🧠 Convert pixel_values back into tensors
    if batch[0]["pixel_values"] is not None:
        pixel_values = torch.stack([
            torch.tensor(b["pixel_values"]) for b in batch
        ])
    else:
        pixel_values = None

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "pixel_values": pixel_values
    }

In [ ]:
%pip -q install peft

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Training

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="./gemma3-lora-scienceqa",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_steps=20,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_evalset,
    data_collator=multimodal_collator
)

In [ ]:
%pip -q install bitsandbytes

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("gemma3-scienceqa-lora")

In [ ]:
#in case you wanna load again
# from peft import PeftModel

# base_model = Gemma3ForConditionalGeneration.from_pretrained(...)
# model = PeftModel.from_pretrained(base_model, "gemma3-scienceqa-lora")

In [ ]:
#sanity check
messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are a helpful assistant."}
        ]
    },
    {
        "role": "user", "content": [
            {"type": "text", "text": "In one word: what is the colour of the sky?"}
        ]
    },
]
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    add_generation_prompt=True,
).to("cuda")

output = model.generate(**inputs, max_new_tokens=1, eos_token_id=processor.tokenizer.eos_token_id,pad_token_id=processor.tokenizer.pad_token_id, cache_implementation="static")
print(processor.decode(output[0], skip_special_tokens=True))

In [ ]:
#sanity check
messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are a helpful assitant."}
        ]
    },
    {
        "role": "user", "content": [
            {"type": "text", "text": "What do these two changes have in common? cooking a pancake an old sandwich rotting in a trashcan Choose the best option: 0. Both are caused by cooling 1. Both are chemical changes 2. Both are caused by heating 3.Both are only physical changes\nAnswer:"}
        ]
    },
]
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    add_generation_prompt=True,
).to("cuda")

output = model.generate(**inputs, max_new_tokens=1, eos_token_id=processor.tokenizer.eos_token_id,pad_token_id=processor.tokenizer.pad_token_id, cache_implementation="static")
print(processor.decode(output[0], skip_special_tokens=True))

In [ ]:
print(type(dataset[0].get('image')))

<class 'PIL.PngImagePlugin.PngImageFile'>


In [ ]:
#sanity test
# Step 1: Format the raw ScienceQA sample
image = dataset[0].get('image')
image = image.convert("RGB")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "What is in this image?"}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    add_generation_prompt=True,
).to("cuda")

output = model.generate(**inputs, max_new_tokens=50, cache_implementation="static")
print(processor.decode(output[0], skip_special_tokens=True))

CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.


user




What is in this image?
model
Certainly! 

The image is a map of the United States. Here's what's visible:

*   **States:** The map shows the boundaries of several states, including Arizona, Oklahoma, Louisiana, and West Virginia.
*
